<a href="https://colab.research.google.com/github/annasysoeva/Computational-Linguistics/blob/main/%D0%A1%D1%8B%D1%81%D0%BE%D0%B5%D0%B2%D0%B0__%22w2v_hw_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В этом практикуме мы рассмотрим работу с библиотекой **Gensim** для работы с векторными представлениями текста

Мы рассмотрим
- **Word2Vec** - векторные представления слов
- **FastText** - улучшенные представления с учетом морфологии  
- **Doc2Vec** - векторные представления документов


In [1]:
!pip install gensim

import gensim
import gensim.downloader as api
from gensim.models import Word2Vec, FastText, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 34.6 MB/s eta 0:00:00


## Часть 1: Word2Vec

### Что такое Word2Vec?

Word2Vec преобразует слова в векторы чисел так, что семантически похожие слова оказываются близко в векторном пространстве.

**Два основных алгоритма:**
- **CBOW** - предсказывает слово по контексту
- **Skip-gram** - предсказывает контекст по слову

**Загрузка предобученной модели**

In [ ]:
w2v_model = api.load('glove-wiki-gigaword-100')

print(f"Размер словаря: {len(w2v_model.key_to_index)}")
print(f"Размерность векторов: {w2v_model.vector_size}")

[==================================================] 100.0% 128.1/128.1MB downloaded


Найдите документацию `gensim`: какие датасеты кроме `glove-wiki-gigaword-100` доступны в библиотеке?

Выберите 3 датасета и кратко опишите их (источник данных, примерный объем, зачем такой датасет может использоваться)

**Базовые операции с векторами**

In [ ]:
# Получаем вектор слова
vector = w2v_model['computer']
print(f"Вектор слова 'computer': {vector[:5]}...")  # Показываем первые 5 чисел

# Вычисляем схожесть между словами
similarity = w2v_model.similarity('computer', 'laptop')
print(f"Схожесть 'computer' и 'laptop': {similarity:.4f}")

**Поиск похожих слов**

In [ ]:
# Находим похожие слова
similar_words = w2v_model.most_similar('python', topn=5)
print("Слова, похожие на 'python':")
for word, score in similar_words:
    print(f"  {word}: {score:.4f}")

*Ваш ответ здесь*

**Задание**

1. Загрузите любой датасет из gensim на ваш выбор

In [1]:
import gensim.downloader as api

# Загрузка модели Word2Vec для английского языка
wv_model = api.load('word2vec-google-news-300')

# Выводим размер словаря и размерность векторов
print(f"Размер словаря (английский): {len(wv_model.key_to_index)}")
print(f"Размерность векторов (английский): {wv_model.vector_size}")

# Получаем 5 самых похожих слов к "king"
similar_words = wv_model.most_similar('king', topn=5)
print("\nПохожие слова к 'king':")
print(similar_words)

# Получаем вектор для слова "queen"
queen_vector = wv_model['queen']
print("\nВектор для слова 'queen':")
print(queen_vector)

# Операции с векторами: король - мужчина + женщина = королева
result = wv_model.most_similar(positive=['king', 'woman'], negative=['man'], topn=1)
print("\nРезультат операции 'king - man + woman':")
print(result)

# Загрузка модели Word2Vec для русского языка
rus_model = api.load('word2vec-ruscorpora-300')

# Выводим размер словаря и размерность векторов для русского языка
print(f"\nРазмер словаря (русский): {len(rus_model.key_to_index)}")
print(f"Размерность векторов (русский): {rus_model.vector_size}")

# Получаем 5 самых похожих слов к "король"
similar_rus_words = rus_model.most_similar('король_NOUN', topn=5) # Изменено на 'король_NOUN'
print("\nПохожие слова к 'король':")
print(similar_rus_words)


Размер словаря (английский): 3000000
Размерность векторов (английский): 300

Похожие слова к 'king':
[('kings', 0.7138045430183411), ('queen', 0.6510956883430481), ('monarch', 0.6413194537162781), ('crown_prince', 0.6204220056533813), ('prince', 0.6159993410110474)]

Вектор для слова 'queen':
[ 0.00524902 -0.14355469 -0.06933594  0.12353516  0.13183594 -0.08886719
 -0.07128906 -0.21679688 -0.19726562  0.05566406 -0.07568359 -0.38085938
  0.10400391 -0.00081635  0.1328125   0.11279297  0.07275391 -0.046875
  0.06591797  0.09423828  0.19042969  0.13671875 -0.23632812 -0.11865234
  0.06542969 -0.05322266 -0.30859375  0.09179688  0.18847656 -0.16699219
 -0.15625    -0.13085938 -0.08251953  0.21289062 -0.35546875 -0.13183594
  0.09619141  0.26367188 -0.09472656  0.18359375  0.10693359 -0.41601562
  0.26953125 -0.02770996  0.17578125 -0.11279297 -0.00411987  0.14550781
  0.15625     0.26757812 -0.01794434  0.09863281  0.05297852 -0.03125
 -0.16308594 -0.05810547 -0.34375    -0.17285156  0.11

2. Напишите функцию, которая принимает на вход любое слово и вовращает 10 наиболее близких по вектору слов

In [2]:
import gensim.downloader as api

# Загрузка модели Word2Vec для английского языка
wv_model = api.load('word2vec-google-news-300')

def find_similar_words(word, model, topn=10):
    """
    Функция для нахождения наиболее похожих слов по вектору.

    :param word: Слово, для которого ищем похожие.
    :param model: Модель Gensim, которая имеет предобученные векторы.
    :param topn: Количество самых похожих слов (по умолчанию 10).
    :return: Список из кортежей (слово, схожесть).
    """
    try:
        # Находим и возвращаем topn наиболее похожих слов
        return model.most_similar(word, topn=topn)
    except KeyError:
        # В случае ошибки (например, если слова нет в словаре модели)
        return f"Слово '{word}' не найдено в модели."

# Пример использования
word = 'king'  # Можно заменить на любое слово
similar_words = find_similar_words(word, wv_model)

print(f"10 наиболее похожих слов к '{word}':")
for similar_word, similarity in similar_words:
    print(f"{similar_word}: {similarity}")


10 наиболее похожих слов к 'king':
kings: 0.7138045430183411
queen: 0.6510956883430481
monarch: 0.6413194537162781
crown_prince: 0.6204220056533813
prince: 0.6159993410110474
sultan: 0.5864824056625366
ruler: 0.5797567367553711
princes: 0.5646552443504333
Prince_Paras: 0.5432944297790527
throne: 0.5422105193138123


3. Обучите модель Word2Vec на тестовом датасете из ячейки ниже

Примените следующие настройки:

- размер вектора: 50
- размер окна: 3
- минимальная частота слова: 1
- потоков: 2
- использовать skip-gram

In [3]:
cooking_sentences = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

In [4]:
import gensim
from gensim.models import Word2Vec

# Тестовый датасет
cooking_sentences = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

# Обучение модели Word2Vec
model = Word2Vec(sentences=cooking_sentences,
                 vector_size=50,         # Размерность векторов
                 window=3,               # Размер окна
                 min_count=1,            # Минимальная частота слова
                 workers=2,              # Количество потоков
                 sg=1)                   # Используем Skip-gram (sg=1)

# Сохранение обученной модели
model.save("cooking_word2vec.model")

# Пример использования модели
# Получаем вектор для слова "суп"
vector_soup = model.wv['варить']
print(f"Вектор для слова 'варить':\n{vector_soup}\n")

# Находим 10 наиболее похожих слов к слову "варить"
similar_words = model.wv.most_similar('варить', topn=10)
print("10 наиболее похожих слов к 'варить':")
for similar_word, similarity in similar_words:
    print(f"{similar_word}: {similarity}")


Вектор для слова 'варить':
[ 0.00855824  0.00014491 -0.01917186 -0.0193191  -0.01229258 -0.00025519
  0.0040027   0.01887415  0.01115754 -0.00859665  0.0005563   0.00992073
  0.01539659 -0.00228468  0.00863558 -0.01160553 -0.00160849  0.01618568
 -0.00471722 -0.01933858  0.01157407 -0.00784942 -0.00244693  0.01995833
 -0.00450418 -0.00950285 -0.01066121  0.01396874 -0.01142338  0.00423821
 -0.01051207  0.01224589  0.00872661  0.00520848 -0.00298893 -0.00547967
  0.01799387  0.01044352 -0.00433021 -0.01893173 -0.01484578 -0.0021372
 -0.00160586 -0.00512253  0.01936917 -0.00091621  0.01175271 -0.01490094
 -0.0050148  -0.01109139]

10 наиболее похожих слов к 'варить':
вино: 0.23975302278995514
ингредиенты: 0.21724146604537964
хлеб: 0.1937645971775055
брокколи: 0.18456541001796722
кипятить: 0.17112019658088684
огурцы: 0.16630668938159943
холодильник: 0.16596782207489014
тост: 0.14339452981948853
тесто: 0.13644389808177948
овощи: 0.1319844275712967


In [5]:
print(f"Слова в словаре: {list(model.wv.key_to_index.keys())[:10]}...")

Слова в словаре: ['овощи', 'мясо', 'соус', 'вода', 'тесто', 'духовка', 'специи', 'варить', 'брокколи', 'питание']...


4. Проверьте модель

In [6]:
# Проверяем похожие слова в кулинарной тематике
try:
    similar = model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

Слова, похожие на 'варить':
  вино: 0.2398
  ингредиенты: 0.2172
  хлеб: 0.1938
  брокколи: 0.1846
  кипятить: 0.1711


In [7]:
# Проверяем похожие слова в кулинарной тематике
try:
    # Для слова "варить"
    similar = model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

# Найдите слова, похожие на "духовка"
try:
    similar_oven = model.wv.most_similar('духовка', topn=5)
    print("\nСлова, похожие на 'духовка':")
    for word, score in similar_oven:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'духовка' не найдено в словаре")

# Найдите слова, похожие на "овощи"
try:
    similar_vegetables = model.wv.most_similar('овощи', topn=5)
    print("\nСлова, похожие на 'овощи':")
    for word, score in similar_vegetables:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'овощи' не найдено в словаре")


Слова, похожие на 'варить':
  вино: 0.2398
  ингредиенты: 0.2172
  хлеб: 0.1938
  брокколи: 0.1846
  кипятить: 0.1711

Слова, похожие на 'духовка':
  ингредиенты: 0.3199
  десерт: 0.3064
  холодильник: 0.2705
  питание: 0.2243
  пирог: 0.2142

Слова, похожие на 'овощи':
  мариновать: 0.2716
  хлеб: 0.2691
  гриль: 0.2546
  фольга: 0.2409
  сахар: 0.2108


## Часть 2: FastText

FastText улучшает Word2Vec, рассматривая слова как наборы символов (n-грамм). Это позволяет работать с редкими словами и опечатками

5. Обучите FastText на корпусе текстов из пункта 3. Используйте код ниже

In [ ]:
ft_model = FastText(
    sentences=sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2
)

6. Найдите слова, похожие на "варить", "духовка" и "овощи" с помощью обученной модели. Используйте код из пункта 4

In [8]:
from gensim.models import FastText

# Тестовый датасет
cooking_sentences = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

# Обучение модели FastText
ft_model = FastText(
    sentences=cooking_sentences,
    vector_size=50,  # Размерность вектора
    window=3,        # Размер окна
    min_count=1,     # Минимальная частота слова
    workers=2        # Количество потоков
)

# Сохранение обученной модели
ft_model.save("cooking_fasttext.model")

# Функция для нахождения похожих слов
def find_similar_words_ft(word, model, topn=5):
    try:
        # Находим и возвращаем topn наиболее похожих слов
        return model.wv.most_similar(word, topn=topn)
    except KeyError:
        return f"Слово '{word}' не найдено в словаре."

# Проверяем похожие слова в кулинарной тематике с помощью модели FastText
# Для слова "варить"
similar_words_varit = find_similar_words_ft('варить', ft_model)
print("Слова, похожие на 'варить':")
for word, score in similar_words_varit:
    print(f"  {word}: {score:.4f}")

# Для слова "духовка"
similar_words_duhovka = find_similar_words_ft('духовка', ft_model)
print("\nСлова, похожие на 'духовка':")
for word, score in similar_words_duhovka:
    print(f"  {word}: {score:.4f}")

# Для слова "овощи"
similar_words_ovoshi = find_similar_words_ft('овощи', ft_model)
print("\nСлова, похожие на 'овощи':")
for word, score in similar_words_ovoshi:
    print(f"  {word}: {score:.4f}")


Слова, похожие на 'варить':
  жарить: 0.5353
  парить: 0.4805
  месить: 0.3541
  тушить: 0.3405
  специи: 0.2622

Слова, похожие на 'духовка':
  взбивать: 0.4565
  лимон: 0.3561
  салат: 0.3050
  курица: 0.3041
  тост: 0.2944

Слова, похожие на 'овощи':
  жарить: 0.2960
  фольга: 0.2574
  морковь: 0.2297
  соус: 0.2172
  торт: 0.2094


7. Сравните модели

Дана функция для сравнения Word2Vec и FastText

Придумайте 3 слова с опечатками и проверьте, найдет ли их FastText и Word2Vec

In [9]:
# Функция для сравнения моделей Word2Vec и FastText
def compare_models(word):
    """Сравнивает представления слова в разных моделях"""
    print(f"\nСравнение для слова: '{word}'")

    # Word2Vec
    try:
        w2v_similar = model.wv.most_similar(word, topn=2)
        print(f"  Word2Vec: {[w for w, _ in w2v_similar]}")
    except KeyError:
        print(f"  Word2Vec: слово не найдено")

    # FastText
    try:
        ft_similar = ft_model.wv.most_similar(word, topn=2)
        print(f"  FastText: {[w for w, _ in ft_similar]}")
    except KeyError:
        print(f"  FastText: слово не найдено")

# Сравниваем для разных слов, включая слова с опечатками
compare_models('learining')  # Опечатка в "learning"
compare_models('neurla')     # Опечатка в "neural"
compare_models('neaural')    # Опечатка в "neural"



Сравнение для слова: 'learining'
  Word2Vec: слово не найдено
  FastText: ['картофель', 'тост']

Сравнение для слова: 'neurla'
  Word2Vec: слово не найдено
  FastText: ['барбекю', 'рыба']

Сравнение для слова: 'neaural'
  Word2Vec: слово не найдено
  FastText: ['вино', 'здоровое']


## Часть 3: Doc2Vec

Doc2Vec расширяет Word2Vec для создания векторных представлений целых документов (предложений, абзацев, статей)

In [10]:
# Создаем размеченные документы
documents = [
    "machine learning is interesting",
    "deep learning uses neural networks",
    "python programming for data science",
    "artificial intelligence is amazing",
    "computer vision processes images"
]

# Преобразуем в формат TaggedDocument
tagged_docs = []
for i, doc in enumerate(documents):
    tokens = doc.split()
    tagged_doc = TaggedDocument(words=tokens, tags=[f"doc_{i}"])
    tagged_docs.append(tagged_doc)

print("Размеченные документы:")
for doc in tagged_docs[:3]:
    print(f"  Слова: {doc.words}")
    print(f"  Тег: {doc.tags}")

NameError: name 'TaggedDocument' is not defined

In [12]:
# Обучаем Doc2Vec
doc_model = Doc2Vec(
    documents=tagged_docs,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    epochs=20
)

print("Doc2Vec модель обучена!")
print(f"Количество документов: {len(doc_model.dv.key_to_index)}")

NameError: name 'Doc2Vec' is not defined

In [11]:
# Получаем вектор документа
doc_vector = doc_model.dv["doc_0"]
print(f"Вектор документа doc_0: {doc_vector[:5]}...")

# Находим похожие документы
similar_docs = doc_model.dv.most_similar("doc_0", topn=2)
print("\nДокументы, похожие на doc_0:")
for doc_tag, similarity in similar_docs:
    doc_id = int(doc_tag.split('_')[1])
    print(f"  {doc_tag}: {similarity:.4f}")
    print(f"    Текст: {documents[doc_id]}")

NameError: name 'doc_model' is not defined

In [13]:
# Сравниваем схожесть документов
def compare_documents(doc1_id, doc2_id):
    similarity = doc_model.dv.similarity(f"doc_{doc1_id}", f"doc_{doc2_id}")
    print(f"Схожесть doc_{doc1_id} и doc_{doc2_id}: {similarity:.4f}")
    print(f"  doc_{doc1_id}: {documents[doc1_id]}")
    print(f"  doc_{doc2_id}: {documents[doc2_id]}")

compare_documents(0, 1)  # machine learning vs deep learning
compare_documents(0, 3)  # machine learning vs AI

NameError: name 'doc_model' is not defined

8. Сравните схожесть doc_2 и doc_4

In [14]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

# 1. Создаем размеченные документы
documents = [
    "machine learning is interesting",
    "deep learning uses neural networks",
    "python programming for data science",
    "artificial intelligence is amazing",
    "computer vision processes images"
]

# Преобразуем в формат TaggedDocument
tagged_docs = []
for i, doc in enumerate(documents):
    tokens = doc.split()
    tagged_doc = TaggedDocument(words=tokens, tags=[f"doc_{i}"])
    tagged_docs.append(tagged_doc)

print("Размеченные документы:")
for doc in tagged_docs[:3]:
    print(f"  Слова: {doc.words}")
    print(f"  Тег: {doc.tags}")

# 2. Обучаем Doc2Vec
doc_model = Doc2Vec(
    documents=tagged_docs,
    vector_size=50,  # Размерность вектора
    window=3,        # Размер окна
    min_count=1,     # Минимальная частота слова
    workers=2,       # Количество потоков
    epochs=20        # Количество эпох
)

print("Doc2Vec модель обучена!")
print(f"Количество документов: {len(doc_model.dv.key_to_index)}")

# 3. Получаем вектор документа
doc_vector = doc_model.dv["doc_0"]
print(f"Вектор документа doc_0: {doc_vector[:5]}...")

# 4. Находим похожие документы
similar_docs = doc_model.dv.most_similar("doc_0", topn=2)
print("\nДокументы, похожие на doc_0:")
for doc_tag, similarity in similar_docs:
    doc_id = int(doc_tag.split('_')[1])
    print(f"  {doc_tag}: {similarity:.4f}")
    print(f"    Текст: {documents[doc_id]}")

# 5. Сравниваем схожесть документов
def compare_documents(doc1_id, doc2_id):
    """Сравнивает схожесть двух документов по их векторным представлениям."""
    similarity = doc_model.dv.similarity(f"doc_{doc1_id}", f"doc_{doc2_id}")
    print(f"Схожесть doc_{doc1_id} и doc_{doc2_id}: {similarity:.4f}")
    print(f"  doc_{doc1_id}: {documents[doc1_id]}")
    print(f"  doc_{doc2_id}: {documents[doc2_id]}")

# Сравниваем схожесть документов
compare_documents(0, 1)  # machine learning vs deep learning
compare_documents(0, 3)  # machine learning vs AI


Размеченные документы:
  Слова: ['machine', 'learning', 'is', 'interesting']
  Тег: ['doc_0']
  Слова: ['deep', 'learning', 'uses', 'neural', 'networks']
  Тег: ['doc_1']
  Слова: ['python', 'programming', 'for', 'data', 'science']
  Тег: ['doc_2']
Doc2Vec модель обучена!
Количество документов: 5
Вектор документа doc_0: [-0.01057    -0.01198188 -0.01982618  0.01710627  0.00710373]...

Документы, похожие на doc_0:
  doc_1: 0.2735
    Текст: deep learning uses neural networks
  doc_2: 0.1275
    Текст: python programming for data science
Схожесть doc_0 и doc_1: 0.2735
  doc_0: machine learning is interesting
  doc_1: deep learning uses neural networks
Схожесть doc_0 и doc_3: -0.0822
  doc_0: machine learning is interesting
  doc_3: artificial intelligence is amazing


9. Найдите самый похожий документ на doc_1

In [15]:
# Находим самый похожий документ на doc_1
similar_docs_to_doc1 = doc_model.dv.most_similar("doc_1", topn=1)

print("\nДокумент, похожий на doc_1:")
for doc_tag, similarity in similar_docs_to_doc1:
    doc_id = int(doc_tag.split('_')[1])
    print(f"  {doc_tag}: {similarity:.4f}")
    print(f"    Текст: {documents[doc_id]}")



Документ, похожий на doc_1:
  doc_0: 0.2735
    Текст: machine learning is interesting


10. Выберите любую из трёх моделей. Обучите модели с разной размерностью (10, 50, 100). Продемонстрируйте качество их работы на примере поиска похожих слов (выберите любые 3 примера, соответствующих тематике корпуса из пункта 4)

In [16]:
from gensim.models import FastText

# Тестовый датасет (кулинарная тематика)
cooking_sentences = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

# Список для тестирования
words_to_check = ['варить', 'духовка', 'овощи']

# Функция для обучения и поиска похожих слов
def train_and_find_similar(size):
    print(f"\nОбучаем модель FastText с размерностью {size}...")

    # Обучение модели FastText
    ft_model = FastText(
        sentences=cooking_sentences,
        vector_size=size,  # Размерность вектора
        window=3,          # Размер окна
        min_count=1,       # Минимальная частота слова
        workers=2          # Количество потоков
    )

    # Находим похожие слова для каждого из выбранных
    for word in words_to_check:
        print(f"\nПохожие слова к '{word}':")
        try:
            similar_words = ft_model.wv.most_similar(word, topn=5)
            for similar_word, score in similar_words:
                print(f"  {similar_word}: {score:.4f}")
        except KeyError:
            print(f"  Слово '{word}' не найдено в словаре")

# Тестирование модели с разной размерностью
for size in [10, 50, 100]:
    train_and_find_similar(size)



Обучаем модель FastText с размерностью 10...

Похожие слова к 'варить':
  готовить: 0.7547
  парить: 0.7291
  травы: 0.6609
  мука: 0.6586
  взбивать: 0.4311

Похожие слова к 'духовка':
  фольга: 0.6091
  тост: 0.6021
  парить: 0.5900
  мука: 0.5387
  суп: 0.5356

Похожие слова к 'овощи':
  парить: 0.7486
  лимон: 0.6758
  питание: 0.5705
  гриль: 0.5695
  духовка: 0.5286

Обучаем модель FastText с размерностью 50...

Похожие слова к 'варить':
  жарить: 0.5353
  парить: 0.4805
  месить: 0.3541
  тушить: 0.3405
  специи: 0.2622

Похожие слова к 'духовка':
  взбивать: 0.4565
  лимон: 0.3561
  салат: 0.3050
  курица: 0.3041
  тост: 0.2944

Похожие слова к 'овощи':
  жарить: 0.2960
  фольга: 0.2574
  морковь: 0.2297
  соус: 0.2172
  торт: 0.2094

Обучаем модель FastText с размерностью 100...

Похожие слова к 'варить':
  жарить: 0.5888
  парить: 0.5608
  пирог: 0.2518
  готовить: 0.2469
  сахар: 0.2383

Похожие слова к 'духовка':
  специи: 0.2608
  мука: 0.2187
  готовить: 0.2125
  паста: 